### Konwersja xml do csv
Dział sensownie poprzez przetwarzanie potokowe poszczególnych plików xml. Oszczędza pamięć RAM i działa w rozsądnym czasie.

In [ ]:
# import danych

import pandas as pd
import numpy as np
import os
import xml.etree.ElementTree as ET
import csv

rootdir = os.path.join(os.getcwd(), "files") # wszystkie xml'e zgrupowane w folderach w folderze files

badges_fields = ["Id", "UserId", "Name", "Date", "Class", "TagBased"]
users_fields = ["Id", "Reputation", "CreationDate", "DisplayName", "LastAccessDate", "WebsiteUrl","Location","AboutMe","Views","UpVotes","DownVotes","AccountId"]
posts_fields = ["Id", "PostTypeId", "AcceptedAnswerId","CreationDate", "Score", "ViewCount", "Body","OwnerUserId","LastEditorUserId","LastEditDate","LastActivityDate","Title","Tags","AnswerCount","CommentCount","FavouriteCount","ParentId"]
comments_fields = ["Id", "PostId", "Score", "Text", "CreationDate", "UserId"]
votes_fields = ["Id", "PostId", "VoteTypeId", "CreationDate"]
postHistory_fields = ["Id", "PostHistoryTypeId", "PostId", "RevisionGUID", "CreationDate", "UserId","Text"]
tags_fields = ["Id", "TagName", "Count", "ExcerptPostId", "WikiPostId"]
postLinks_fields = ["Id", "CreationDate", "PostId", "RelatedPostId", "LinkTypeId"]

In [ ]:
for subdir, dirs, files in os.walk(rootdir):
    for directory in dirs:
        if directory != None and not os.path.exists(directory):
            os.mkdir(directory)
    for file in files:
        filename, _ = os.path.splitext(file)
        folder = os.path.basename(subdir)
        
        csv_fname = folder+"_"+filename+".csv"
        print(csv_fname+" is starting created")
        
        with open(os.path.join(folder, csv_fname), mode="w", encoding="utf-8") as csvfile:
            
            if filename == "Posts":
                fieldnames=posts_fields
                writer = csv.DictWriter(csvfile, fieldnames=posts_fields)
            if filename == "Badges":
                fieldnames=badges_fields
                writer = csv.DictWriter(csvfile, fieldnames=badges_fields)
            if filename == "Users":
                fieldnames=users_fields
                writer = csv.DictWriter(csvfile, fieldnames=users_fields)
            if filename == "Tags":
                fieldnames=tags_fields
                writer = csv.DictWriter(csvfile, fieldnames=tags_fields)
            if filename == "Votes":
                fieldnames=votes_fields
                writer = csv.DictWriter(csvfile, fieldnames=votes_fields)
            if filename == "Comments":
                fieldnames=comments_fields
                writer = csv.DictWriter(csvfile, fieldnames=comments_fields)
            if filename == "PostHistory":
                fieldnames=postHistory_fields
                writer = csv.DictWriter(csvfile, fieldnames=postHistory_fields)    
            if filename == "PostLinks":
                fieldnames=postLinks_fields
                writer = csv.DictWriter(csvfile, fieldnames=postLinks_fields)    
            writer.writeheader()
            
            xml = os.path.join(subdir, file)
            context = ET.iterparse(xml)
            
            filled_row = {}
            context = iter(context)
            i=0
            k=0
            for event, elem in context:
                if elem.tag == 'row':
                    for head in fieldnames:
                        try:
                            filled_row.update({head : elem.attrib[head]})
                        except:
                            filled_row.update({head : np.nan})        
                    writer.writerow(filled_row)
                    i=i+1;
                    if i>100000:
                        print(k, end=" ")
                        k=k+1
                        i=0
                elem.clear()
                del elem
        print(csv_fname+" created")
        